In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/firebaseserviceaccountkey/sahayak2-5ed9b-firebase-adminsdk-fbsvc-0320b2f8a4.json


In [2]:
# ==============================
# 1️⃣ Install Required Packages
# ==============================
!pip install -q fastapi uvicorn transformers accelerate bitsandbytes pyngrok firebase-admin

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 19.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 37.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 37.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 3.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 5.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# ==============================
# 1. Imports and Secrets
# ==============================
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
from pyngrok import ngrok
import uvicorn
import threading
import time # <--- Import the time library
from kaggle_secrets import UserSecretsClient

# Get secrets from Kaggle
user_secrets = UserSecretsClient()
secret_hf_token = user_secrets.get_secret("HUGGING_FACE_TOKEN")
secret_ngrok_token = user_secrets.get_secret("NGROK_AUTH_TOKEN")

# ==============================
# 2. Model Loading (4-bit)
# ==============================
MODEL_PATH = "shl0k/Sahayak-Llama-3-8B-Instruct-v1"

print("⏳ Loading Sahayak model...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, token=secret_hf_token)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    device_map="auto",
    torch_dtype=torch.float16,
    load_in_4bit=True,
    token=secret_hf_token
)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
print("✅ Sahayak model loaded successfully.")

# ==============================
# 3. FastAPI App
# ==============================
app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

class Item(BaseModel):
    prompt: str

@app.post("/generate")
def generate_text(item: Item):
    prompt = item.prompt
    if not prompt:
        return {"response": "Error: No prompt provided."}

    print(f"📥 Prompt received: {prompt[:80]}...")
    output = pipe(prompt, max_new_tokens=250, temperature=0.7, do_sample=True)
    model_response = output[0]['generated_text']
    print("✅ Response generated.")
    
    return {"response": model_response}

# ==============================
# 4. Start ngrok & Server
# ==============================
ngrok.set_auth_token(secret_ngrok_token)

def run_server():
    uvicorn.run(app, host="0.0.0.0", port=8000)

thread = threading.Thread(target=run_server)
thread.start()

public_url = ngrok.connect(8000)
print("====================================================================")
print("🚀 Your Sahayak Model is Live!")
print(f"🔗 Public API URL: {public_url}")
print("====================================================================")

# --- START OF FIX ---
# Keep the main script running indefinitely to keep the server alive.
# This loop will pause the script for 60 seconds at a time, forever.
while True:
    time.sleep(60)
# --- END OF FIX ---

2025-08-09 12:24:43.983122: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754742284.164763      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754742284.215421      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


⏳ Loading Sahayak model...


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

...

📥 Prompt received: You are Sahayak, an expert AI assistant that creates simple analogies using Indi...
✅ Response generated.
INFO:     223.185.134.147:0 - "POST /generate HTTP/1.1" 200 OK


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


📥 Prompt received: You are Sahayak, an expert AI assistant that creates simple analogies using Indi...
✅ Response generated.
INFO:     223.185.134.147:0 - "POST /generate HTTP/1.1" 200 OK


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


📥 Prompt received: You are Sahayak, an expert AI assistant that creates simple analogies using Indi...
✅ Response generated.
INFO:     223.185.134.147:0 - "POST /generate HTTP/1.1" 200 OK
